In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
#from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
#from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow


%matplotlib inline

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator


data_generator = ImageDataGenerator(rescale=1./255, validation_split = 0.3)

train_generator = data_generator.flow_from_directory('C:/Users/dhruv/OneDrive/Desktop/7th sem project/tomato/train', subset="training")
valid_generator = data_generator.flow_from_directory('C:/Users/dhruv/OneDrive/Desktop/7th sem project/tomato/val', subset="validation")

In [ ]:
print(train_generator.__len__())
print(train_generator[0].__len__())
print(train_generator[0][0].__len__())
print(train_generator[0][0][0].__len__())
print(train_generator[0][0][0][0].__len__())
print(train_generator[0][0][0][0][0].__len__())

print(train_generator[0][0][0][0][0]) # 1 pixel

In [ ]:
print(valid_generator.__len__())
print(valid_generator[0].__len__())
print(valid_generator[0][0].__len__())
print(valid_generator[0][0][0].__len__())
print(valid_generator[0][0][0][0].__len__())
print(valid_generator[0][0][0][0][0].__len__())

print(valid_generator[0][0][0][0][0]) # 1 pixel

In [ ]:
x_train, y_train = train_generator.next()
print(x_train.__len__(), y_train.__len__())
print(x_train[0].__len__(), y_train[0].__len__())


In [ ]:
import matplotlib.pyplot as plt
print(x_train.shape, x_train[0].shape)
plt.imshow(x_train[0])
plt.show()

In [ ]:
resnetv2 = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top=False,
    input_shape=(256,256,3)
)

resnetv2.trainable = False

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomContrast(0.15),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
model = Sequential([
    data_augmentation,
    resnetv2,
    Dense(512,activation='relu'),
    BatchNormalization(),
    GlobalAveragePooling2D(), # No global - > shape :(None, 4,4,10)
    # Global for Flatten
    Dense(10,activation='softmax')
])

In [ ]:
model.compile( optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(
        train_generator,
        batch_size=32,
        epochs=10,
        validation_data=valid_generator,
        validation_steps=800
)

In [ ]:
test_generator = data_generator.flow_from_directory('C:/Users/dhruv/OneDrive/Desktop/7th sem project/tomato/val')

loss, accuracy = model.evaluate(test_generator)
print("Loss is ", loss) # Epoch : 8
print("Accuracy is ", accuracy) # With more Contrast and Rotation
# Contrast : 0.5, Rotate = 0.5

In [ ]:
from tensorflow.keras.preprocessing import image

# Load the image
img_path = r'C:\Users\dhruv\OneDrive\Desktop\7th sem project\tomato\Images\35797a1e-c933-4ae3-817f-abc07775075d___GCREC_Bact.Sp_3204-removebg-preview.png'
img = image.load_img(img_path, target_size=(256, 256))

# Preprocess the image
img_array = image.img_to_array(img)
plt.imshow(img_array/255.)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Rescale to match the rescaling done during training

# Get predictions
predictions = model.predict(img_array)

# Interpret predictions
class_names = train_generator.class_indices
predicted_class = np.argmax(predictions)
predicted_label = list(class_names.keys())[predicted_class]
print("Predicted class:", predicted_label)

